In [2]:
import os
import glob
import pandas as pd
import numpy as np

path = '/Users/baffoebonnieja/Downloads'

subdirs = glob.glob(f'{path}/*onsets.csv')
print(subdirs)

for x in subdirs: 
    print(x)

['/Users/baffoebonnieja/Downloads/scene_onsets.csv', '/Users/baffoebonnieja/Downloads/food_onsets.csv', '/Users/baffoebonnieja/Downloads/S4_onsets.csv']
/Users/baffoebonnieja/Downloads/scene_onsets.csv
/Users/baffoebonnieja/Downloads/food_onsets.csv
/Users/baffoebonnieja/Downloads/S4_onsets.csv


In [2]:
##FINAL 

import os
import glob
import pandas as pd
import numpy as np

path = '/System/Volumes/Data/misc/data17/persichettias/words_fmri/'
subdirs = glob.glob(f'{path}words20/expInfo/wordsfmri_20*') #adds subdirs of interest to empty list

sorted_subdirs = sorted(subdirs)
sorted_subdirs

output_dict = {
    'word': [],
    'run': [],
    'onset_time': [],
    'duration': [],
    'reaction_time': [],
    'button_press': []
}

for dir in sorted_subdirs:
    run = os.path.basename(dir).split('_')[2][:-4]
    df = pd.read_csv(dir)
    ID = os.path.basename(dir).split('_')[1]
    df.rename(columns={'words.started': 'word.started', 'words.stopped': 'word.stopped'}, inplace=True)
    
    df['onset_time'] = df['word.started'] - df['buffer.started'].iloc[2]
    df['duration'] = df['word.stopped'] - df['word.started']
    
    for i in range(len(df)):
        output_dict['run'].append(run)
        output_dict['word'].append(df['word'].iloc[i])
        output_dict['duration'].append(df['duration'].iloc[i])
        output_dict['reaction_time'].append(df['response.rt'].iloc[i])
        output_dict['button_press'].append(df['response.keys'].iloc[i])
        output_dict['onset_time'].append(df['onset_time'].iloc[i])
#dict to df
output_df = pd.DataFrame(output_dict)

output_df = output_df[output_df['word'].notna()]
output_df = output_df[output_df["word"].str.strip() != ""]  # removes rows where "word" is empty


#check values before replacing 
print("button_press unique values:\n", output_df['button_press'].unique())
print("old button_press types:\n", output_df['button_press'].apply(type).value_counts()) #classifies and counts data types

# replace button_press      SWITCHED : subj19
output_df['button_press'] = output_df['button_press'].replace({'b': 1, 'g': 2, None: 0, np.nan: 0, 'None': 0, 'none': 0, 'NA': 0, 'na': 0, 'N/A': 0, 'n/a': 0, 'nan': 0, 'NaN': 0, 'null': 0})
output_df['button_press'] = pd.to_numeric(output_df['button_press'], errors='coerce').fillna(0).astype(int)

#change missing RT to 0
output_df['reaction_time'] = pd.to_numeric(output_df['reaction_time'], errors='coerce').fillna(0)

# Debug
print("corrected button_press unique values:\n", output_df['button_press'].unique())
print("corrected button_press types:\n", output_df['button_press'].apply(type).value_counts()) #classifies and counts data types

output_df

output_df.to_csv(f"/System/Volumes/Data/misc/data17/persichettias/words_fmri/behavior/words{ID}_scan_data.csv", index=False)


#output_df.to_csv(f"/Users/baffoebonnieja/Downloads/words{ID}_scan_data.csv", index=False)


button_press unique values:
 ['g' 'b' 'None']
old button_press types:
 <class 'str'>    536
Name: button_press, dtype: int64
corrected button_press unique values:
 [2 1 0]
corrected button_press types:
 <class 'int'>    536
Name: button_press, dtype: int64


In [5]:
output_df

,word,run,onset_time,duration,reaction_time,button_press
3,diet,1,4.010070,5.991007,1.779340,2
4,mountain,1,10.002564,5.998579,0.697410,1
5,rebel,1,16.001788,6.003890,1.422105,2
6,football,1,22.006831,6.000871,0.702803,1
7,perception,1,28.009244,5.990113,0.945517,2
...,...,...,...,...,...,...
562,key,8,411.986800,5.999003,0.946570,1
563,distribution,8,417.986573,6.000754,1.276415,2
564,butterfly,8,423.988089,6.013039,0.689889,1
565,opposition,8,430.002037,5.990250,1.779765,2


In [93]:
##SCORING SORTED psychopy data 
##make sure abswords_key.csv is in the same folder

import pandas as pd
import os

os.chdir('/Users/baffoebonnieja/Library/CloudStorage/OneDrive-NationalInstitutesofHealth/PsychoPy_Script')

key_df = pd.read_csv('abswords_key.csv') 
#replace
word_df = pd.read_csv('words5_scan_data.csv')

word_df['Accuracy'] = ' '

if len(word_df['button press']) != len(key_df['answer']):
    raise ValueError("The number of entries in word_df and key_df must match.")
    
for i in range(len(word_df)):
        # Check if the current button press matches answer in key_df
    if word_df['button press'].iloc[i] == key_df['answer'].iloc[i]:  
        word_df.at[i, 'Accuracy'] = 2
        
    elif word_df['button press'].iloc[i] == '**':
        word_df.at[i, 'Accuracy'] = 0
        
    else:
        word_df.at[i, 'Accuracy'] = 1
        
        

word_df.to_csv('/Users/baffoebonnieja/Downloads/words5_scored.csv', index = 'False')

#Outputs list of 0s 1s and 2s


FileNotFoundError: [Errno 2] No such file or directory: 'abswords_key.csv'